In [1]:
import requests
import json
import pandas as pd

In [2]:
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": [
    'APU0000706111', 'APU0000704111', 'APU0000711211', 'APU0000702111', 'APU0000717311', 'APU0000708111', 'APU0000701111'
    , 'APU0000709112', 'APU0000711311', 'APU0000701312', 'APU0000712311', 'APU000072610', 'APU000072511'
    , 'APU00007471A', 'APU000074714'
    ]
        ,"startyear":"2012", "endyear":"2022", "registrationkey": "6a4fc3836e344229b9fe9dc5925c842e",
       "catalog": "true", "annualaverage": "true", "calculations": "true"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)

In [3]:
json_data['Results']['series'][0]['data'][0]

{'year': '2022',
 'period': 'M11',
 'periodName': 'November',
 'latest': 'true',
 'value': '1.843',
 'footnotes': [{}],
 'calculations': {'net_changes': {'1': '-0.020',
   '3': '-0.036',
   '6': '0.019',
   '12': '0.260'},
  'pct_changes': {'1': '-1.1', '3': '-1.9', '6': '1.0', '12': '16.4'}}}

In [4]:
json_data['Results']['series'][0]['catalog']

{'series_title': 'Chicken, fresh, whole, per lb. (453.6 gm) in U.S. city average, average price, not seasonally adjusted',
 'series_id': 'APU0000706111',
 'survey_name': 'CPI Average Price Data, U.S. city average (AP)',
 'survey_abbreviation': 'AP',
 'measure_data_type': 'Chicken, fresh, whole, per lb. (453.6 gm)',
 'area': 'U.S. city average',
 'item': 'Chicken, fresh, whole, per lb. (453.6 gm)'}

## I determined the key errors to be happening because of some missing keys in the json data for some of the "net_changes"; I removed these from my final python script and opted to perform these calcs in PowerQuery.

In [12]:
for a in json_data['Results']['series']:
    series_title = a['catalog']['series_title']
    series_id = a['catalog']['series_id']
    survey_name = a['catalog']['survey_name']
    survey_abbreviation = a['catalog']['survey_abbreviation']
    measure_data_type = a['catalog']['measure_data_type']
    area = a['catalog']['area']
    for b in a['data']:
        print(a['catalog']['series_id'])
        print(b['calculations']['net_changes']['1'])

APU0000706111
-0.020
APU0000706111
-0.028
APU0000706111
0.012
APU0000706111
-0.001
APU0000706111
0.054
APU0000706111
0.002
APU0000706111
0.030
APU0000706111
0.070
APU0000706111
0.092
APU0000706111
0.010
APU0000706111
0.016
APU0000706111
0.023
APU0000706111
0.060
APU0000706111
0.019
APU0000706111
0.032
APU0000706111
0.037
APU0000706111
-0.039
APU0000706111
-0.012
APU0000706111
-0.029
APU0000706111
-0.028
APU0000706111
-0.040
APU0000706111
-0.012
APU0000706111
-0.026
APU0000706111
-0.018
APU0000706111
0.059
APU0000706111
0.038
APU0000706111
-0.067
APU0000706111
-0.103
APU0000706111
-0.035
APU0000706111


KeyError: '1'

## Identifying specific arrays that are missing the 4 keys I expected (expecting 1, 3, 6, and 12):

In [15]:
for i in json_data['Results']['series'][0]['data']:
    if len(i['calculations']['net_changes']) != 4:
        print(i['calculations']['net_changes'])

{'1': '-0.029', '3': '-0.097', '6': '-0.153'}
{'1': '0.059', '3': '0.030', '12': '0.209'}
{'1': '-0.103', '6': '0.247', '12': '0.095'}
{'3': '0.347', '6': '0.297', '12': '0.157'}


In [17]:
df = pd.DataFrame(columns=[
    'series_title', 'series_id', 'survey_name', 'survey_abbreviation', 'measure_data_type', 'area'
    , 'year', 'period', 'period_name', 'value'
                          ])

## Determined that percent change calculations will be done in PowerBI since they are not always provided in the data, so those datapoints are commented out below:

In [20]:
for a in json_data['Results']['series']:
    series_title = a['catalog']['series_title']
    series_id = a['catalog']['series_id']
    survey_name = a['catalog']['survey_name']
    survey_abbreviation = a['catalog']['survey_abbreviation']
    measure_data_type = str(a['catalog']['measure_data_type']).replace("Fuel ", "")
    area = a['catalog']['area']
    for b in a['data']:
        year = b['year']
        period = b['period']
        period_name = b['periodName']
        value = b['value']
        #prev_per_net_chg = b['calculations']['net_changes']['1']
        #three_per_net_chg = b['calculations']['net_changes']['3']
        #six_per_net_chg = b['calculations']['net_changes']['6']
        #twelve_per_net_chg = b['calculations']['net_changes']['12']
        #prev_per_pct_chg = b['calculations']['pct_changes']['1']
        #three_per_pct_chg = b['calculations']['pct_changes']['3']
        #six_per_pct_chg = b['calculations']['pct_changes']['6']
        #twelve_per_pct_chg = b['calculations']['pct_changes']['12']

        #save data in dataframe
        df = pd.concat([df, pd.DataFrame.from_records([{
            'series_title': series_title
            , 'series_id': series_id
            , 'survey_name': survey_name
            , 'survey_abbreviation': survey_abbreviation
            , 'measure_data_type': measure_data_type
            , 'area': area
            , 'year': year
            , 'period': period
            , 'period_name': period_name
            , 'value': value
            #, 'prev_per_net_chg': prev_per_net_chg
            #, 'three_per_net_chg': three_per_net_chg
            #, 'six_per_net_chg': six_per_net_chg
            #, 'twelve_per_net_chg': twelve_per_net_chg
            #, 'prev_per_pct_chg': prev_per_pct_chg
            #, 'three_per_pct_chg': three_per_pct_chg
            #, 'six_per_pct_chg': six_per_pct_chg
            #, 'twelve_per_pct_chg': twelve_per_pct_chg
        }])], ignore_index=True)

In [19]:
print(df)

                                           series_title      series_id  \
0     Chicken, fresh, whole, per lb. (453.6 gm) in U...  APU0000706111   
1     Chicken, fresh, whole, per lb. (453.6 gm) in U...  APU0000706111   
2     Chicken, fresh, whole, per lb. (453.6 gm) in U...  APU0000706111   
3     Chicken, fresh, whole, per lb. (453.6 gm) in U...  APU0000706111   
4     Chicken, fresh, whole, per lb. (453.6 gm) in U...  APU0000706111   
...                                                 ...            ...   
1929  Gasoline, unleaded regular, per gallon/3.785 l...   APU000074714   
1930  Gasoline, unleaded regular, per gallon/3.785 l...   APU000074714   
1931  Gasoline, unleaded regular, per gallon/3.785 l...   APU000074714   
1932  Gasoline, unleaded regular, per gallon/3.785 l...   APU000074714   
1933  Gasoline, unleaded regular, per gallon/3.785 l...   APU000074714   

                                         survey_name survey_abbreviation  \
0     CPI Average Price Data, U.S. 